In [ ]:
import pandas as pd
import numpy as np
import folium
from geopy.geocoders import Nominatim
from datetime import datetime, timedelta

# Load the dataset
file_path = '/content/Aggregated_Routes_Dataset.csv'
data = pd.read_csv(file_path)


data['Latitude'] = data['Latitude'].apply(eval)
data['Longitude'] = data['Longitude'].apply(eval)

# User inputs for start and destination points
print("Enter the coordinates for the start and destination points:")
start_lat = float(input("Start Latitude: "))
start_lon = float(input("Start Longitude: "))
end_lat = float(input("Destination Latitude: "))
end_lon = float(input("Destination Longitude: "))


def find_nearest_route(data, start_lat, start_lon):
    distances = []
    for i, row in data.iterrows():
        first_point = (row['Latitude'][0], row['Longitude'][0])  # First point of each route
        distance = np.sqrt((start_lat - first_point[0])**2 + (start_lon - first_point[1])**2)
        distances.append(distance)
    return np.argmin(distances)

nearest_route_idx = find_nearest_route(data, start_lat, start_lon)
nearest_route = data.iloc[nearest_route_idx]


route_latitudes = nearest_route['Latitude']
route_longitudes = nearest_route['Longitude']


def generate_vector_field(route_latitudes, route_longitudes, destination, grid_size=100):
    """
    Generate vector field guiding towards the destination while following the route.
    """
    dest_lat, dest_lon = destination


    lat_min, lat_max = min(route_latitudes), max(route_latitudes)
    lon_min, lon_max = min(route_longitudes), max(route_longitudes)
    lat_grid, lon_grid = np.linspace(lat_min, lat_max, grid_size), np.linspace(lon_min, lon_max, grid_size)
    X, Y = np.meshgrid(lon_grid, lat_grid)


    U, V = np.zeros_like(X), np.zeros_like(Y)


    for lat, lon in zip(route_latitudes, route_longitudes):
        dx = dest_lon - lon
        dy = dest_lat - lat
        distance = np.sqrt(dx**2 + dy**2)
        U += dx / (distance + 1e-5)
        V += dy / (distance + 1e-5)

    # Normalize vectors
    magnitude = np.sqrt(U**2 + V**2)
    U, V = U / magnitude, V / magnitude

    return X, Y, U, V

destination = (end_lat, end_lon)
X, Y, U, V = generate_vector_field(route_latitudes, route_longitudes, destination)


geolocator = Nominatim(user_agent="ship_route_mapper")
def get_place_name(lat, lon):
    try:
        location = geolocator.reverse((lat, lon), exactly_one=True)
        return location.address.split(",")[0] if location else "Unknown"
    except:
        return "Unknown"

start_place = get_place_name(start_lat, start_lon)
end_place = get_place_name(end_lat, end_lon)

# Calculate travel time (assume constant average speed)
average_speed_kmh = 20  # Example average speed in km/h
distance_km = np.sum(
    np.sqrt(
        np.diff(route_latitudes)**2 + np.diff(route_longitudes)**2
    )
) * 111  # Approx. km per degree latitude/longitude

travel_time_hours = distance_km / average_speed_kmh
start_time = datetime.now()
arrival_time = start_time + timedelta(hours=travel_time_hours)

#  Plot the map with Folium

center_lat, center_lon = np.mean(route_latitudes), np.mean(route_longitudes)
ship_map = folium.Map(location=[center_lat, center_lon], zoom_start=6)


folium.PolyLine(list(zip(route_latitudes, route_longitudes)), color='blue', weight=3, opacity=0.7).add_to(ship_map)


folium.Marker(
    [start_lat, start_lon],
    popup=f"Start: {start_place}\nTime: {start_time.strftime('%Y-%m-%d %H:%M:%S')}",
    icon=folium.Icon(color='green')
).add_to(ship_map)

folium.Marker(
    [end_lat, end_lon],
    popup=f"Destination: {end_place}\nTime: {arrival_time.strftime('%Y-%m-%d %H:%M:%S')}",
    icon=folium.Icon(color='red')
).add_to(ship_map)


map_path = '/content/optimal_ship_route_with_vector_field.html'
ship_map.save(map_path)

print(f"Optimal ship route plotted and saved to: {map_path}")


Enter the coordinates for the start and destination points:
Start Latitude: 23.0360
Start Longitude: 70.2239
Destination Latitude: 25.2769
Destination Longitude: 55.2962
Optimal ship route plotted and saved to: /content/optimal_ship_route_with_vector_field.html
